# Cell Segmentation

This notebook uses SAM to segment the given images and stores the segmentation information in a csv file for each image

## Segment Anything Model (SAM)

Download checkpoint files here: https://pypi.org/project/segment-anything-py/#model-checkpoints

This models requieres 16 GB of RAM (or VRAM) to work.


### Imports

In [ ]:
import os
import sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" # If you have more than one GPU, use this to select the one you want to use


sys.path.insert(0, "../../packages/python")
from data import utils as data_utils
from models import cell_segmentation as segmentators

2025-06-08 16:03:44.915582: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-08 16:03:44.922444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749409424.930410  164600 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749409424.932754  164600 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749409424.938991  164600 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("cuDNN Enabled:", tf.test.is_built_with_cuda())

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
cuDNN Enabled: True


### Global definitions

In [3]:
sys.path.insert(0, "../../")
from config import MODELS_PATH, DATASETS_PATH

# Specific paths
SAM_CHECKPOINT_PATH = os.path.join(MODELS_PATH, "SAM/sam_vit_h_4b8939.pth")

# Select the devicce: 
# "cuda" : Will use the NVIDIA GPU
# "cpu" : Will use the... CPU
DEVICE_USE = "cuda"

### Load SAM model

In [4]:
cmg = segmentators.SAMCellMaskGenerator(SAM_CHECKPOINT_PATH, model_type = 'vit_h', device = DEVICE_USE)

### Apply segmentation to the whole
Using SAM to segment the images in IMAGE_PATH the output will be stored in separeted csv files in CSV_PATH

In [ ]:
# Dataset selection
# DATASET = 'ina'#'ina' # Change this to 'onion_cell_merged' if needed
# DATASET_SECTION = '' # Change this to 'train', 'valida' or 'test' for 'onion_cell_merged', empty for 'ina'
DATASET = 'onion_cell_merged'#'ina' # Change this to 'onion_cell_merged' if needed
DATASET_SECTION = 'test' # Change this to 'train', 'valida' or 'test' for 'onion_cell_merged', empty for 'ina'

IMAGE_PATH = os.path.join(DATASETS_PATH, 'full_fov', DATASET, 'images', DATASET_SECTION)
CSV_PATH = os.path.join(DATASETS_PATH, 'cropped', DATASET, 'data', DATASET_SECTION)

In [6]:
# Apply segmentation to the whole
data_utils.dataset_cell_segmentation(cmg, IMAGE_PATH, CSV_PATH)

100%|██████████| 63/63 [06:42<00:00,  6.38s/it]


In [7]:
CSV_PATH

'/home/rawthil/Documents/GIAR/INA/INA_code/giar_ina_dev/datasets/onion_cell_merged/data/test'